In [175]:
import collections
import copy
import itertools

import cspUtil
import dataUtil

In [73]:
def countSequences(inputList, N, normalize=True):
    out = collections.defaultdict(float)
    for i in range(len(inputList)-N+1):
        if [inputList[i+k] != None for k in range(N)] == [True for _ in range(N)]:
            seq = tuple([inputList[i+k] for k in range(N)])
            out[seq] += 1
    
    if normalize:
        factor= 1.0 / sum(out.itervalues())
        for k in out:
            out[k] *= 10 ** N * factor
            
    return out

In [1]:
def createRythmCSP(windowedPrimTrack, bi=True, tri=False):
    cspRythm = cspUtil.CSP()

    # Creating "units" variables
    transitions = [note[1] for note in windowedPrimTrack]
    for unitNum in range(windowStart, windowEnd+1):
        unit = windowedPrimTrack[unitNum]
        cspRythm.add_variable(('U', unitNum), [(unit[0],True),(unit[0],False)])

    if bi:
        # Create binary factors proportional to frequence of apparition of pairs
        biSequences = countSequences(transitions, 2)
        print "bi"
        print biSequences
        for varNum in range(windowStart, windowEnd):
            cspRythm.add_binary_factor(('U', varNum), ('U', varNum+1), lambda u1, u2 : biSequences[(u1[1], u2[1])])
        # Add corner cases
        cspRythm.add_unary_factor(('U', windowStart), lambda u : biSequences[(windowedPrimTrack[windowStart-1][1], u[1])])
        cspRythm.add_unary_factor(('U', windowEnd)  , lambda u : biSequences[(u[1], windowedPrimTrack[windowEnd+1][1])])

    if tri:
        # Creating "binary" variables for 3-sequences handling
        domain = [(False, False), (False, True), (True, False), (True, True)]
        for unitNum in range(windowStart, windowEnd+1):
            cspRythm.add_variable(('B', unitNum), copy.deepcopy(domain))

        # Creating binary factors proportional to frequence of apparition of triads
        triSequences = countSequences(transitions, 3)

        for varNum in range(windowStart, windowEnd+1):
            # Add constraint B_i[0] = U_i[1]
            cspRythm.add_binary_factor(('U', varNum), ('B', varNum), lambda u, b: u[1] == b[0])

            if varNum != windowEnd:
                # Add consitency contraints B_i[0] = B_{i+1}[1] and frequency factor
                cspRythm.add_binary_factor(('B', varNum), ('B', varNum+1), \
                                           lambda b1, b2: (b1[0] == b2[1]) * triSequences[(b1[1], b1[0], b2[0])])
        # Add corner cases
        cspRythm.add_unary_factor(('B', windowStart), lambda b: b[1] == windowedPrimTrack[windowStart-1])
        cspRythm.add_unary_factor(('B', windowEnd)  , \
                lambda b: triSequences[(b[1], b[0], windowedPrimTrack[windowEnd+1])] \
                        * triSequences[(b[0], windowedPrimTrack[windowEnd+1], windowedPrimTrack[windowEnd+2])])

    return cspRythm

In [1]:
def basicPitchCSP(pitchTrack, window, verbose=True):
    csp = cspUtil.CSP()
    start, end = window
    
    # Compute domain as set of existing notes in track #
    domain = set()
    for pitch in pitchTrack:
        if pitch is not None: domain.add(pitch)
    domain = tuple(domain)
    
    # Create variables which represent missing pitches as ('U', numVar) #
    for pitchIdx in range(start, end):
        csp.add_variable(('U', pitchIdx), copy.copy(domain))
            
    # Adding factors which choose pitch depending on frequency of single apparition #
    freq1 = countSequences(pitchTrack, 1)
    for pitchIdx in range(start, end):
        break
        csp.add_unary_factor(('U', pitchIdx), lambda u : freq1[(u,)])
        
    ## Adding factors which choose pair of pitch depending on frequence of pair apparition ##
    freq2 = countSequences(pitchTrack, 2)
    for pitchIdx in range(start, end-1):
        break
        csp.add_binary_factor(('U', pitchIdx), ('U', pitchIdx+1), lambda u1, u2 : freq2[(u1, u2)])
        
    ### Compute domain for variables which represent two consecutive pitches ###
    domain2 = tuple(itertools.product(domain, repeat = 2))
    
    ### Create variables which represent 2 consecutive pitches as ('B', pitchIdx) ###
    for pitchIdx in range(start, end):
        csp.add_variable(('B', pitchIdx), copy.deepcopy(domain2))
        
    ### Creating binary factors for link between P1 and P2: B_i[0] = U_i ### 
    for pitchIdx in range(start, end):
             csp.add_binary_factor(('U', pitchIdx), ('B', pitchIdx), lambda u, b: b[0] == u)
                                                                                                                                                                                                     # Creating "binary" variables for 3-sequences handling
    ### Creating binary factors proportional to frequence of apparition of triads ###                                                                                                                               triSequences = countSequences(windowedPrimTrack, 3)
    freq3 = countSequences(pitchTrack, 3, True)
    for pitchIdx in range(start, end):
        # Add constraint B_i[0] = U_i
        csp.add_binary_factor(('U', pitchIdx), ('B', pitchIdx), lambda u, b: u == b[0])
        
        if pitchIdx < end-1:
            # Add consitency contraints B_i[0] = B_{i+1}[1] and frequency factor
            csp.add_binary_factor(('B', pitchIdx), ('B', pitchIdx+1), \
                                    lambda b1, b2: (b1[0] == b2[1]) * freq3[(b1[1], b1[0], b2[0])])
    # Add corner cases
    csp.add_unary_factor(('B', start), lambda b: b[1] == pitchTrack[start-1] \
                          * freq3[(pitchTrack[start-2], pitchTrack[start-1], b[0])])
    csp.add_unary_factor(('B', end-1), lambda b: freq3[(b[1], b[0], pitchTrack[end+1])] \
                     * freq3[(b[0], pitchTrack[end+1], pitchTrack[end+2])])
        
    if verbose:
        print "Domain:", domain
        print "Variables:", csp.variables
        print "Unary factors:", csp.unaryFactors
        print "Binary factors:", csp.binaryFactors
        print "freq3:", freq3

    return csp

In [84]:
idx = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
ode = [4, 4, 5, 7, 7, 5, 4, 2, 0, 0, 2, 4, 4, 2, 4, 4, 5, 7, 7, 5, 4, 2, 0, 0, 2, 4, 2, 0, 0]
odeCorrupt = copy.copy(ode)
start, end = 15, 18
for i in range(start, end):
    odeCorrupt[i] = None
csp = basicPitchCSP(odeCorrupt, (start, end), verbose=True)
search = cspUtil.BacktrackingSearch()
search.solve(csp)

Domain: (0, 2, 4, 5, 7)
Variables: [('U', 15), ('U', 16), ('U', 17), ('B', 15), ('B', 16), ('B', 17)]
Unary factors: {('B', 15): {(4, 7): 0.0, (7, 7): 0.0, (0, 7): 0.0, (2, 5): 0.0, (7, 2): 0.0, (4, 0): 0.0, (5, 5): 0.0, (4, 4): 0.0, (5, 0): 1.0, (2, 2): 0.0, (5, 4): 0.0, (0, 0): 1.0, (4, 5): 0.0, (0, 4): 0.0, (7, 5): 0.0, (0, 5): 0.0, (4, 2): 0.0, (2, 7): 0.0, (7, 0): 1.0, (5, 7): 0.0, (0, 2): 0.0, (7, 4): 0.0, (2, 0): 0.0, (5, 2): 0.0, (2, 4): 0.0}, ('B', 16): None, ('U', 15): None, ('B', 17): {(4, 7): 0.0, (7, 7): 4132.231404958678, (0, 7): 0.0, (2, 5): 0.0, (7, 2): 0.0, (4, 0): 0.0, (5, 5): 0.0, (4, 4): 0.0, (5, 0): 0.0, (2, 2): 0.0, (5, 4): 0.0, (0, 0): 0.0, (4, 5): 0.0, (0, 4): 0.0, (7, 5): 0.0, (0, 5): 0.0, (4, 2): 0.0, (2, 7): 0.0, (7, 0): 0.0, (5, 7): 0.0, (0, 2): 0.0, (7, 4): 0.0, (2, 0): 0.0, (5, 2): 0.0, (2, 4): 0.0}, ('U', 17): None, ('U', 16): None}
Binary factors: {('B', 15): {('B', 16): {(4, 7): {(4, 7): 0.0, (7, 7): 0.0, (0, 7): 0.0, (2, 5): 0.0, (7, 2): 0.0, (4, 0): 0

In [89]:
freq2 = countSequences(odeCorrupt, 2)
freq2

defaultdict(int,
            {(0, 0): 3,
             (0, 2): 2,
             (2, 0): 3,
             (2, 4): 2,
             (4, 2): 3,
             (4, 4): 1,
             (4, 5): 2,
             (5, 4): 2,
             (5, 7): 2,
             (7, 5): 2,
             (7, 7): 2})

In [179]:
dom = (1, 2, 4)
print tuple(itertools.product(dom, repeat = 2))


((1, 1), (1, 2), (1, 4), (2, 1), (2, 2), (2, 4), (4, 1), (4, 2), (4, 4))


In [ ]:
  # Creating "units" variables #
    # Domains
    minPitch, maxPitch = pitchTrack[0], pitchTrack[0]
    for pitch in pitchTrack:
        if type(pitch) == int:
            minPitch, maxPitch = min(minPitch, pitch), max(maxPitch, pitch)
    domains = tuple(list(range(minPitch, maxPitch+1)) + ['silence'])
    for pitchNum in range(windowStart, windowEnd+1):
        pitch = pitchTrack[pitchNum]
        cspPitch.add_variable(('U', pitchNum), copy.deepcopy(domains))

    # Create binary factors proportional to frequence of apparition of pairs
    biSequences = countSequences(pitchTrack, 2)
    print "bi"
    print biSequences
    for varNum in range(windowStart, windowEnd):
        cspPitch.add_binary_factor(('U', varNum), ('U', varNum+1), lambda p1, p2 : biSequences[(p1, p2)])
    # Add corner cases
    cspPitch.add_unary_factor(('U', windowStart), lambda p : biSequences[(pitchTrack[windowStart-1], p)])
    cspPitch.add_unary_factor(('U', windowEnd)  , lambda p : biSequences[(p, pitchTrack[windowEnd+1])])

    """
    if triSeq:
        # Creating "binary" variables for 3-sequences handling
        domain = [(False, False), (False, True), (True, False), (True, True)]
        for unitNum in range(windowStart, windowEnd+1):
            cspPitch.add_variable(('B', unitNum), copy.deepcopy(domain))

        # Creating binary factors proportional to frequence of apparition of triads
        triSequences = countSequences(windowedPrimTrack, 3)

        for varNum in range(windowStart, windowEnd+1):
            # Add constraint B_i[0] = U_i[1]
            cspPitch.add_binary_factor(('U', varNum), ('B', varNum), lambda u, b: u[1] == b[0])

            if varNum != windowEnd:
                # Add consitency contraints B_i[0] = B_{i+1}[1] and frequency factor
                cspPitch.add_binary_factor(('B', varNum), ('B', varNum+1), \
                                           lambda b1, b2: (b1[0] == b2[1]) * triSequences[(b1[1], b1[0], b2[0])])
        # Add corner cases
        cspPitch.add_unary_factor(('B', windowStart), lambda b: b[1] == windowedPrimTrack[windowStart-1])
        cspPitch.add_unary_factor(('B', windowEnd)  , \
                lambda b: triSequences[(b[1], b[0], windowedPrimTrack[windowEnd+1])] \
                        * triSequences[(b[0], windowedPrimTrack[windowEnd+1], windowedPrimTrack[windowEnd+2])])
        """

    return cspPitch

In [5]:
midiFile = "../../test/ode.mid"
windowStart, windowEnd = 10, 15
ode = dataUtil.MusicPattern(midiFile)
rythmUnit, windowedPrimTrack = ode.getCorrupt(windowStart, windowEnd)
pitchTrack = [note[0] for note in windowedPrimTrack]
CSP = createRythmCSP(windowedPrimTrack, bi=False, tri=True)
print windowedPrimTrack[(windowStart-1):(windowEnd+2)]
print CSP.variables
print CSP.values
print "unary"
print CSP.unaryFactors
print "binary"
print CSP.binaryFactors
search = cspUtil.BacktrackingSearch()
search.solve(CSP)

Loaded ../../test/ode.mid


AttributeError: MusicPattern instance has no attribute 'rythmUnits'